In [8]:
# version in online jupyter notebook: 0.26.1
#!pip install openai==0.26.1

!pip list

Package                   Version
------------------------- --------------
accelerate                0.26.0
aiohappyeyeballs          2.5.0
aiohttp                   3.11.13
aiosignal                 1.3.2
annotated-types           0.7.0
anyio                     4.8.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
datasets                  3.2.0
debugpy                   1.8.13
decorator                 5.2.1
defusedxml                0.7.1
dill                      0.3.8
distro                    1.9.0
executing     


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
if 'A306709' in os.environ['USERNAME']:
    print("Running on Christophs computer: update proxy settings.")
    os.environ["http_proxy"] = "http://sia-lb.telekom.de:8080"
    os.environ["https_proxy"] = "http://sia-lb.telekom.de:8080"
else:
    print("Running on any computer but not Christophs: don't update any proxy settings.")

Running on Christophs computer: update proxy settings.


In [10]:
import openai

import json
import os
import time

#print(openai.version)
#print(openai.__version__)

In [11]:
openai.api_key =  os.getenv("OPENAI_API_KEY")
openai.api_base = "https://openai.vocareum.com/v1" # Remove this if using personal key
#print(openai.api_key)

In [12]:
# Decoding parameters
TEMPERATURE = 0.0
MAX_TOKENS = 3950  # Increased to simulate LLM with smaller attention window
TOP_P = 1.0

In [13]:
SYSTEM_PROMPT = """You expert at games of chance.
End every response with double exclamation points!!"""

USER_NAME = "User"
AI_NAME = "AI Assistant"
NEW_INTERACTION_DELIMITER = "\n\n"

# Creating a chat bot with memory
Using the basic `openai.Completion` API to understand where the memory "lives" in the prompt.

In [14]:
def query_openai(prompt):
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        top_p=TOP_P,
    )
    time.sleep(5) # to avoid rate limit errors
    if "error" in response:
        #raise openai.APIError(response["error"], param=None)
        raise openai.InvalidRequestError(response["error"], param=None)
    else:
        return response.choices[0].text.strip().strip("\n")


def get_system_prompt(input_str=SYSTEM_PROMPT):
    return [f"System:{input_str}"]


def get_convo(input_str, convo):
    if not convo:
        convo = get_system_prompt()
    user_input_str = f"{USER_NAME}: {input_str}"
    response_trigger = f"{AI_NAME}: "
    convo.extend([user_input_str, response_trigger])
    return convo


# This is the function we will be fixing
def get_response(input_str, convo, use_simple_truncation, verbose):
    """
    Generate a response from an LLM based on user input_str and conversation history.

    Parameters:
    input_str (str): The user's current input_str or query to the language model.
    convo (list of str): A list representing the history of the conversation.
    use_simple_truncation (bool): A flag to determine whether to use a simple truncation
                                  method for managing conversation length.
    verbose (bool): A flag to determine if entire convo history should be printed.

    Returns:
    str: The generated response from the language model based on the current input_str and
         the conversation history.
    """
    convo = get_convo(input_str, convo)

    # Try to prompt model and catch if the prompt exceeds the attention window
    first_try = True
    atten_window_all_used_up = False
    while first_try or atten_window_all_used_up:
        # Convo list flattened into string to feed to model
        flattened_convo = NEW_INTERACTION_DELIMITER.join(convo)
        #flattened_convo = "" # TODO
        #cut_front = False
        #for current_str in convo:
        #    flattened_convo += ", " + current_str
        #    cut_front = True
        #if cut_front:
        #    flattened_convo = flattened_convo[2:]

        try:
            first_try = False
            response = query_openai(flattened_convo)
            atten_window_all_used_up = False

        except openai.InvalidRequestError as e:
        #except openai.APIError as e:
            atten_window_all_used_up = True
            if verbose:
                print("** ATTEN_WINDOW ALL USED UP **")
                print(f"OpenAI Error: {repr(e)}\n")

            if not convo:
                return [
                    "Our Error: System prompt is using up too many tokens of the attention window"
                ]

            # We can recover from over-allocation of atten_window by removing 
            # components from history.
            if use_simple_truncation:
                # Just remove oldest element in convo
                if len(convo) > 0:
                    convo = convo[1:]  # TODO

            else:
                # Remove the oldest User or AI convo turn, while retaining 
                # system prompt
                if len(convo) > 1:
                    convo = convo[:1] + convo[2:]  # TODO

    # Add the LLM response to the response_trigger
    convo[-1] += response
    if verbose:
        print(NEW_INTERACTION_DELIMITER.join(convo))
    else:
        print(f"{USER_NAME}: {input_str}")
        print(f"{AI_NAME}: {response}")

    return convo


def chat(user_query, convo=[], use_simple_truncation=False, verbose=False):
    convo = get_response(user_query, convo, use_simple_truncation, verbose)
    return convo

## Testing our Chat bot

In [15]:
user_inputs = [
    "What cards game has the best odds of winning?",
    "What are the odds of winning it?",
    "What is the best hand to be dealt?",
    "What is the next most likely game to win?",
]

### Simple convo truncation method
Model remembers prior convo but **forgets to** always ends in exclamation points!!

In [16]:
convo = []
verbose = False
simple_truncation = True
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(
        input, convo=convo, use_simple_truncation=simple_truncation, verbose=verbose
    )
    print()

**** Convo turn 0 ****
User: What cards game has the best odds of winning?
AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
User: What are the odds of winning it?
AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
User: What is the best hand to be dealt?
AI Assistant: The best hand to be dealt in blackjack is a natural blackjack, which is an ace and a 10-value card. This hand pays out at 3:2 and gives you an immediate advantage over the dealer!!

**** Convo turn 3 ****
User: What is the next most likely game to win?
AI Assistant: The next most likely game to win would depend on the specific casino and its games. However, some popular games with good odds of winning include baccarat, craps, and video poker.

#### What is happening?
Peaking under hood to see the full conversation as it grows

NOTE: In **convo turn 3** we remove the system prompt to avoid exceeding the LLM attention window


In [17]:
convo = []
verbose = True
simple_truncation = True
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(
        input, convo=convo, use_simple_truncation=simple_truncation, verbose=verbose
    )
    print()

**** Convo turn 0 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

User: What are the odds of winning it?

AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game 

### Retaining System prompt truncation method
Model remembers most of prior convo and **remembers to** always ends in exclamation points!!

In [18]:
convo = []
verbose = False
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(input, convo=convo, verbose=verbose)
    print()

**** Convo turn 0 ****
User: What cards game has the best odds of winning?
AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
User: What are the odds of winning it?
AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
User: What is the best hand to be dealt?
AI Assistant: The best hand to be dealt in blackjack is a natural blackjack, which is an ace and a 10-value card. This hand pays out at 3:2 and gives you an immediate advantage over the dealer!!

**** Convo turn 3 ****
User: What is the next most likely game to win?
AI Assistant: The next most likely game to win would depend on your personal preferences and skills. Some popular games of chance include poker, roulette, and baccarat. However, with your exper

#### What is happening?
Peaking under hood to see full convo as it grows

NOTE: In **convo turn 3** we remove the oldest **convo turns**, but retain the system prompt

In [19]:
convo = []
verbose = True
for i, input in enumerate(user_inputs):
    print(f"**** Convo turn {i} ****")
    convo = chat(input, convo=convo, verbose=verbose)
    print()

**** Convo turn 0 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

**** Convo turn 1 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game of blackjack has the best odds of winning, with a house edge of only 1%!

User: What are the odds of winning it?

AI Assistant: The odds of winning at blackjack depend on various factors, such as the number of decks in play and the specific rules of the game. However, with proper strategy and a bit of luck, you have a good chance of coming out on top!!

**** Convo turn 2 ****
System:You expert at games of chance.
End every response with double exclamation points!!

User: What cards game has the best odds of winning?

AI Assistant: The game 